In [1]:
import numpy as np
import pandas as pd
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [53]:
df=pd.read_csv("https://raw.githubusercontent.com/manisha-mishra-123/Coursera_Capstone/master/datasets_651191_1152497_df_latlong.csv")
df.head()

,PostalCode,Neighbourhood,Latitude,Longitude
0,122104,Agon,28.46219,77.02373
1,122105,AirForce,28.47762,77.06952
2,122107,Akhera,28.47762,77.06952
3,122001,ArjunNagar,28.45965,77.02004
4,121104,Baded,28.47762,77.06952


In [48]:
address = 'Gurgaon,In'

geolocator = Nominatim(user_agent="in_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Gurgaon are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Gurgaon are 28.42826235, 77.00270014657752.


In [49]:
# create map of New York using latitude and longitude values
map_gurgaon = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, df in zip(df['Latitude'], df['Longitude'],df['Neighbourhood']):
    label = '{}'.format(df)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gurgaon)  
    
map_gurgaon

In [50]:
CLIENT_ID = 'VFEIW4T4YNE1KIEUGTZG1V3TXCJE5DXCGMDURB4ZKFY5RCFJ' # your Foursquare ID
CLIENT_SECRET = 'MYAF32FWZ0QB5N0YXIS1F51G4LGULWIC3XJLE1CN3MTAYT41' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VFEIW4T4YNE1KIEUGTZG1V3TXCJE5DXCGMDURB4ZKFY5RCFJ
CLIENT_SECRET:MYAF32FWZ0QB5N0YXIS1F51G4LGULWIC3XJLE1CN3MTAYT41


In [54]:
df.loc[0, 'Neighbourhood']

'Agon'

In [55]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [56]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
gurgaon_venues = getNearbyVenues(names=df['Neighbourhood'],latitudes=df['Latitude'],longitudes=df['Longitude'],radius=500)

Agon
AirForce
Akhera
ArjunNagar
Baded
Badli
Badshahpur
Baghanki
Basai
BasaiRoad
Baslambi
Baspadamka
Bewan
Bhadas
Bhangrola
Bhirauti
Bhondsi
Bhorakalan
Bichhor
BilaspurKhurd
Bisru
Budarpur
Budhera
Carterpuri
Chakkarpur
Chapera
Cheelarh
DlfQe
Damdama
Daulah
Daultabad
DhaniChitarsain
Dhankot
Doha
Dulawat
Dundahera
Dungerwas
FarrukhNagar
Fazilpur
FerozepurJhirka
FerozepurNamak
GarhiHarsaru
Ghamroj
Ghasera
Ghungola
Gulalta
Gurgaon
GurgaonKty
GurgaonVillage
GwalPahari
Haillymandi
Hasanpur
Hirwari
Inchhapuri
IndirMewat
IndustrialEstate
Jagat
Jaitpur
Jamalgarh
Jamalpur
Janaula
Jant
Jataula
Jatauli
Jawasi
Jhamuwas
Jharsa
Kadipur
Kakoria
Kalwari
Karola
Kasan
Khalilpur
Khandewia
Khandsa
Khatauli
KheraKhurampur
KherkiDaula
Kherla
Khor
Kiranj
Kurthala
Lakhnaula
LalaKherli
LohingaKalan
Lokra
Malab
Mandikhera
Manesar
MaruthiKunj
Mehchana
MiniSectt
Mirpur
MohamadpurAhir
Moklawas
Moolthan
Nagina
Naheda
Nai
Nanukalan
Narhera
Narsinghpur
Nathupur
NewColony
Noorgarh
Nuh
Pachgaon
Pahari
PalamRoad
PalamViha

In [60]:
print(gurgaon_venues.shape)
gurgaon_venues.head()
print('There are {} uniques categories.'.format(len(gurgaon_venues['Venue Category'].unique())))

(478, 7)
There are 83 uniques categories.


In [69]:
gurgaon_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agon,28.46219,77.02373,McDonald's,28.459497,77.026638,Fast Food Restaurant
1,AirForce,28.47762,77.06952,The Westin,28.476890,77.070552,Hotel
2,AirForce,28.47762,77.06952,Prego @ The Westin Gurgaon,28.476926,77.070779,Italian Restaurant
3,AirForce,28.47762,77.06952,The Westin Gurgaon Executive Lounge,28.477479,77.070786,Hotel Bar
4,AirForce,28.47762,77.06952,Eest,28.476949,77.070808,Asian Restaurant


In [95]:
gurgaon_restaurant=gurgaon_venues[gurgaon_venues['Venue Category'].str.contains ("Restaurant")]

gurgaon_restaurant.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agon,28.46219,77.02373,McDonald's,28.459497,77.026638,Fast Food Restaurant
2,AirForce,28.47762,77.06952,Prego @ The Westin Gurgaon,28.476926,77.070779,Italian Restaurant
4,AirForce,28.47762,77.06952,Eest,28.476949,77.070808,Asian Restaurant
8,Akhera,28.47762,77.06952,Prego @ The Westin Gurgaon,28.476926,77.070779,Italian Restaurant
10,Akhera,28.47762,77.06952,Eest,28.476949,77.070808,Asian Restaurant


In [96]:
gurgaon_restaurant.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agon,1,1,1,1,1,1
AirForce,2,2,2,2,2,2
Akhera,2,2,2,2,2,2
Baded,2,2,2,2,2,2
Badshahpur,1,1,1,1,1,1
...,...,...,...,...,...,...
Ujina,2,2,2,2,2,2
Umra,1,1,1,1,1,1
UrbanEstate,1,1,1,1,1,1


In [107]:
gurgaon_grouped = gurgaon_restaurant.groupby('Neighborhood').mean().reset_index()
gurgaon_grouped

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,Agon,28.462190,77.023730,28.459497,77.026638
1,AirForce,28.477620,77.069520,28.476938,77.070793
2,Akhera,28.477620,77.069520,28.476938,77.070793
3,Baded,28.477620,77.069520,28.476938,77.070793
4,Badshahpur,28.393490,77.063390,28.395914,77.059102
...,...,...,...,...,...
60,Ujina,28.477620,77.069520,28.476938,77.070793
61,Umra,28.471040,77.034330,28.469620,77.036283
62,UrbanEstate,28.471131,77.011936,28.473598,77.016203
63,Wazirabad,28.432132,77.091206,28.427957,77.090731


In [108]:
# set number of clusters
kclusters = 3

gurgaon_grouped_clustering = gurgaon_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(gurgaon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int32)

In [109]:
gurgaon_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gurgaon_grouped['Venue Latitude'], gurgaon_grouped['Venue Longitude'], gurgaon_grouped['Neighborhood'], gurgaon_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
a=gurgaon_grouped.loc[gurgaon_grouped['Cluster Labels'] == 0, gurgaon_grouped.columns[[1] + list(range(5, gurgaon_grouped.shape[1]))]]
a

,Neighborhood,Venue Longitude
1,AirForce,77.070793
2,Akhera,77.070793
3,Baded,77.070793
5,Bewan,77.070793
6,Bhadas,77.070793
7,Bhirauti,77.070793
8,Bichhor,77.070793
9,Bisru,77.070793
10,Budarpur,77.070793
11,Carterpuri,77.053861


In [113]:
gurgaon_grouped.loc[gurgaon_grouped['Cluster Labels'] == 1, gurgaon_grouped.columns[[1] + list(range(5, gurgaon_grouped.shape[1]))]]

,Neighborhood,Venue Longitude
52,Sohna,77.067416
53,SohnaAdda,77.067416


In [114]:
gurgaon_grouped.loc[gurgaon_grouped['Cluster Labels'] == 2, gurgaon_grouped.columns[[1] + list(range(5, gurgaon_grouped.shape[1]))]]

,Neighborhood,Venue Longitude
0,Agon,77.026638
4,Badshahpur,77.059102
27,Jharsa,77.044607
28,Kadipur,77.003110
31,KherkiDaula,76.989000
32,Kiranj,77.026638
42,RailwaryRoad,77.016203
43,Ramgarh,77.071171
61,Umra,77.036283
62,UrbanEstate,77.016203
